In [1]:
import pandas as pd
import numpy as np

In [2]:
base = '../results_experiments/'

def extract_best(layer, dataset, opt=None):
    datasets_cpu = ['BZR_MD','COX2_MD', 'ENZYMES']
    datasets_cuda = ['PROTEINS','DD', 'NCI1']

    if dataset in datasets_cpu:
        acellerator = 'cpu'
    else:
        acellerator = 'cuda'

    tmp = base
   
    exp1 = pd.read_csv(f'{tmp}/tu_base_grid_{layer}_{acellerator}/agg/val_best.csv')
    df_val_exp1 = exp1[exp1['dataset'] == dataset]#.sort_values('accuracy', ascending=False)

    exp2 = pd.read_csv(f'{base}/tu_base_grid_{layer}_pos_{acellerator}/agg/val_best.csv')
    df_val_exp2 = exp2[exp2['dataset'] == dataset]#.sort_values('accuracy', ascending=False)

    
    dilated = df_val_exp1[df_val_exp1['type'] == 'dilated_gnn'].sort_values('accuracy', ascending=False)
    std = df_val_exp1[df_val_exp1['type'] == 'standard_gnn'].sort_values('accuracy', ascending=False)
    
    leaf = df_val_exp2.sort_values('accuracy', ascending=False)
    # leaf_len = df_val_exp2[df_val_exp2['pos'] ==  True].sort_values('accuracy', ascending=False)

    df_test_exp1 = pd.read_csv(f'{tmp}/tu_base_grid_{layer}_{acellerator}/agg/test_best.csv')
    df_test_exp2 = pd.read_csv(f'{base}/tu_base_grid_{layer}_pos_{acellerator}/agg/test_best.csv')

    std_test = df_test_exp1[df_test_exp1['model_id'] == std.iloc[0]['model_id']]
    dilated_test = df_test_exp1[df_test_exp1['model_id'] == dilated.iloc[0]['model_id']]
    leaf_test = df_test_exp2[df_test_exp2['model_id'] == leaf.iloc[0]['model_id']]
    # leaf_len_test = df_test_exp2[leaf_len.iloc[0].index]


    if opt == 'values':
    
        res = [
            {
                'layer': layer,
                'dataset': dataset,
                'arc': '2_dilapos',
                'val_accuracy':  f"${round(100*leaf['accuracy'].iloc[0],2)}\pm{round(100*leaf['accuracy_std'].iloc[0],2)}$" ,
                'test_accuracy':  f"${round(100*leaf_test['accuracy'].values[0],2)}\pm{round(100*leaf_test['accuracy_std'].values[0],2)}" if  len(leaf_test['accuracy']) > 0  else 'NA'
            },
            {
                'layer': layer,
                'dataset': dataset,
                'arc': '1_dilated',
                'val_accuracy':  f"${round(100*dilated['accuracy'].iloc[0],2)}\pm{round(100*dilated['accuracy_std'].iloc[0],2)}$" ,
                'test_accuracy':  f"${round(100*dilated_test['accuracy'].values[0],2)}\pm{round(100*dilated_test['accuracy_std'].values[0],2)}$" if  len(dilated_test['accuracy']) > 0  else 'NA'
            },
        
            {
                'layer': layer,
                'dataset': dataset,
                'arc': '0_gnn',
                'val_accuracy':  f"${round(100*std['accuracy'].iloc[0],2)}\pm{round(100*std['accuracy_std'].iloc[0],2)}$" ,
                'test_accuracy': f"${round(100*std_test['accuracy'].values[0],2)}\pm{round(100*std_test['accuracy_std'].values[0],2)}$" if  len(std_test['accuracy']) > 0  else 'NA'
            }
        
        ]
    elif opt == 'stats':
        res = [{
                'layer': layer,
                'dataset': dataset,
                '2_dilapos_val_accuracy':   leaf['accuracy'].iloc[0] ,
                '2_dilapos_test_accuracy': leaf_test['accuracy'].values[0],
           
                '1_dilated_val_accuracy':  dilated['accuracy'].iloc[0] ,
                '1_dilated_test_accuracy':   dilated_test['accuracy'].values[0],
     
                '0_gnn_val_accuracy':  std['accuracy'].iloc[0] ,
                '0_gnn_test_accuracy': std_test['accuracy'].values[0]
            }]
    else:
        res = [
            {
                'layer': layer,
                'dataset': dataset,
                'arc': '2_dilapos',
                'val_accuracy':  f"{round(100*leaf['accuracy'].iloc[0],2)}±{round(100*leaf['accuracy_std'].iloc[0],2)}" ,
                'test_accuracy':  f"{round(100*leaf_test['accuracy'].values[0],2)}±{round(100*leaf_test['accuracy_std'].values[0],2)}" if  len(leaf_test['accuracy']) > 0  else 'NA'
            },
            {
                'layer': layer,
                'dataset': dataset,
                'arc': '1_dilated',
                'val_accuracy':  f"{round(100*dilated['accuracy'].iloc[0],2)}±{round(100*dilated['accuracy_std'].iloc[0],2)}" ,
                'test_accuracy':  f"{round(100*dilated_test['accuracy'].values[0],2)}±{round(100*dilated_test['accuracy_std'].values[0],2)}" if  len(dilated_test['accuracy']) > 0  else 'NA'
            },
        
            {
                'layer': layer,
                'dataset': dataset,
                'arc': '0_gnn',
                'val_accuracy':  f"{round(100*std['accuracy'].iloc[0],2)}±{round(100*std['accuracy_std'].iloc[0],2)}" ,
                'test_accuracy': f"{round(100*std_test['accuracy'].values[0],2)}±{round(100*std_test['accuracy_std'].values[0],2)}" if  len(std_test['accuracy']) > 0  else 'NA'
            }
        
        ]
    return res
    

In [3]:
datasets = ['BZR_MD','COX2_MD', 'ENZYMES', 'PROTEINS','DD', 'NCI1']
layers = ['gin', 'gcn', 'gat',  'sage'] #

res = []
for layer in layers:
    for d in datasets:
        res.extend(extract_best(layer, d, opt='stats'))

res = pd.DataFrame(res).sort_values('layer')

# pd.pivot_table(res, values=['test_accuracy'], 
#                                 index=['layer', 'arc'], 
#                                 columns='dataset', aggfunc='first') 


#res = res[res['dataset'] != 'BZR_MD']
res['best_our_test_accuracy'] = res[['2_dilapos_test_accuracy','1_dilated_test_accuracy' ]].max(axis=1)

res['improvement'] = res['best_our_test_accuracy'] - res['0_gnn_test_accuracy']
res['improvement_2'] = res['2_dilapos_test_accuracy'] - res['0_gnn_test_accuracy']
res['improvement_1'] = res['1_dilated_test_accuracy'] - res['0_gnn_test_accuracy']

print('Mean improvemnt: ', round(res['improvement'].mean()*100,2), '±', round(res['improvement'].std()*100,2))
print('Mean improvement dilapos: ', round(res['improvement_2'].mean()*100,2), '±', round(res['improvement_2'].std()*100,2))
print('Mean improvement dilated: ', round(res['improvement_1'].mean()*100,2), '±', round(res['improvement_1'].std()*100,2))
res[['layer', 'dataset', '0_gnn_test_accuracy', '2_dilapos_test_accuracy','1_dilated_test_accuracy', 'best_our_test_accuracy', 'improvement']]

Mean improvemnt:  1.19 ± 2.26
Mean improvement dilapos:  0.48 ± 2.0
Mean improvement dilated:  1.06 ± 2.28


,layer,dataset,0_gnn_test_accuracy,2_dilapos_test_accuracy,1_dilated_test_accuracy,best_our_test_accuracy,improvement
17,gat,NCI1,0.7959,0.8066,0.7951,0.8066,0.0107
16,gat,DD,0.7282,0.7538,0.7607,0.7607,0.0325
15,gat,PROTEINS,0.7334,0.7406,0.7496,0.7496,0.0162
14,gat,ENZYMES,0.7100,0.7017,0.7200,0.7200,0.0100
13,gat,COX2_MD,0.7333,0.6900,0.6900,0.6900,-0.0433
12,gat,BZR_MD,0.6967,0.6767,0.6767,0.6767,-0.0200
11,gcn,NCI1,0.7392,0.7389,0.7506,0.7506,0.0114
6,gcn,BZR_MD,0.6867,0.7100,0.7100,0.7100,0.0233
7,gcn,COX2_MD,0.6400,0.6367,0.6367,0.6367,-0.0033
8,gcn,ENZYMES,0.5717,0.6050,0.6050,0.6050,0.0333


In [4]:
datasets = ['BZR_MD','COX2_MD', 'ENZYMES', 'PROTEINS', 'NCI1', 'DD']
layers = ['gin', 'gcn', 'gat',  'sage'] #
a = []
for layer in layers:
    res = []
    for d in datasets:
        res.extend(extract_best(layer, d))

    res = pd.DataFrame(res).sort_values('layer')

    a.append(pd.pivot_table(res, values=['test_accuracy'], 
                                    index='dataset', 
                                    columns='arc', aggfunc='first'))

### GIN

In [5]:
a[0]

test_accuracy                        
arc              0_gnn   1_dilated   2_dilapos
dataset                                       
BZR_MD       67.0±8.49  64.33±8.95  64.33±8.95
COX2_MD     63.67±4.82  66.33±4.82  66.33±4.82
DD          72.99±2.48  74.78±3.47   75.9±5.68
ENZYMES      64.0±7.04  65.67±4.96  64.67±4.46
NCI1        80.51±1.87   80.42±1.5   80.68±1.2
PROTEINS     73.6±5.34   75.5±5.16  73.51±4.21

### GNC

In [6]:
a[1] #gcn

test_accuracy                        
arc              0_gnn   1_dilated   2_dilapos
dataset                                       
BZR_MD      68.67±4.52   71.0±8.44   71.0±8.44
COX2_MD      64.0±9.16  63.67±6.23  63.67±6.23
DD            73.5±4.6  75.64±4.82  75.21±4.52
ENZYMES     57.17±6.37   60.5±4.41   60.5±4.41
NCI1        73.92±2.09  75.06±1.99  73.89±2.43
PROTEINS    75.76±5.09  74.23±5.88  74.68±3.29

### GAT
(Due to high requirement of GPU RAM, we didn’t complete a full grid
search, for the model dilated with leafs with GAT on the DD dataset)

In [7]:
a[2]

test_accuracy                        
arc              0_gnn   1_dilated   2_dilapos
dataset                                       
BZR_MD      69.67±8.36  67.67±5.78  67.67±5.78
COX2_MD      73.33±8.3   69.0±6.51   69.0±6.51
DD          72.82±4.06  76.07±5.17  75.38±4.34
ENZYMES      71.0±5.78   72.0±3.78  70.17±6.68
NCI1        79.59±1.58  79.51±1.67  80.66±1.91
PROTEINS    73.34±4.11  74.96±4.75  74.06±4.64

### GraphSAGE

In [8]:
a[3]

test_accuracy                        
arc              0_gnn   1_dilated   2_dilapos
dataset                                       
BZR_MD      75.67±6.51   76.34±9.6   76.34±9.6
COX2_MD      69.0±8.83   72.33±9.2  67.67±6.15
DD          75.64±4.56  76.07±3.99  75.47±2.29
ENZYMES     61.67±6.19  68.33±4.15   66.0±4.84
NCI1        80.95±1.85  81.31±1.84  81.58±1.84
PROTEINS     72.97±3.8  75.14±4.82  73.51±7.29

In [9]:
ogbg_molhiv_val = pd.read_csv(f'../../DL-project_results/ogbg_molhiv_baseline_grid_gin/agg/val_best.csv')
ogbg_molhiv_test = pd.read_csv(f'../../DL-project_results/ogbg_molhiv_baseline_grid_gin/agg/test.csv')

ogbg_molhiv_dilated = ogbg_molhiv_val[ogbg_molhiv_val['type'] == 'dilated_gnn'].sort_values('auc', ascending=False)
ogbg_molhiv_pos = ogbg_molhiv_val[ogbg_molhiv_val['type'] == 'dilapos_gnn'].sort_values('auc', ascending=False)


ogbg_molhiv_dilated_test = ogbg_molhiv_test[ogbg_molhiv_test['model_id'] == ogbg_molhiv_dilated.iloc[0]['model_id']]
ogbg_molhiv_dilapos_test = ogbg_molhiv_test[ogbg_molhiv_test['model_id'] == ogbg_molhiv_pos.iloc[0]['model_id']]

In [10]:
pd.DataFrame([
{
    'layer': 'gin',
    'dataset': 'ogbg-molhiv',
    'arc': 'dilated',
    'val_ROC-AUC':  f"{round(100*ogbg_molhiv_dilated.iloc[0]['auc'],4)}±{round(100*ogbg_molhiv_dilated.iloc[0]['auc_std'],4)}" ,
    'test_ROC-AUC': f"{round(100*ogbg_molhiv_dilated_test.iloc[0]['auc'],4)}±{round(100*ogbg_molhiv_dilated_test.iloc[0]['auc_std'],4)}"
},
{
    'layer': 'gin',
    'dataset': 'ogbg-molhiv',
    'arc': 'dilapos',
    'val_ROC-AUC':  f"{round(100*ogbg_molhiv_pos.iloc[0]['auc'],4)}±{round(100*ogbg_molhiv_pos.iloc[0]['auc_std'],4)}" ,
    'test_ROC-AUC': f"{round(100*ogbg_molhiv_dilapos_test.iloc[0]['auc'],4)}±{round(100*ogbg_molhiv_dilapos_test.iloc[0]['auc_std'],4)}"
},
{
    'layer': 'gin',
    'dataset': 'ogbg-molhiv',
    'arc': 'reference gnn',
    'val_ROC-AUC':  f"82.32 ± 00.90",
    'test_ROC-AUC': f"75.58 ± 01.40"
}
])


,layer,dataset,arc,val_ROC-AUC,test_ROC-AUC
0,gin,ogbg-molhiv,dilated,84.9±1.04,76.56±1.43
1,gin,ogbg-molhiv,dilapos,84.77±0.55,77.49±0.32
2,gin,ogbg-molhiv,reference gnn,82.32 ± 00.90,75.58 ± 01.40
